**Aufgabe 2**

In [4]:
from webencodings.mklabels import assert_lower


def lgt(v, w):
    """Find the longest common subsequence and return the score and the
    backtracking matrix.

    Return (s[n][m], b).
    """

    # Sequence 1
    l_v = [""] + list(v)

    # Sequence 2
    l_w = [""] + list(w)

    n = len(l_v)
    m = len(l_w)

    # Initialize score matrix
    s = [[0 for x in range(m)] for y in range(n)]

    # Initialize backtracking matrix
    b = [["" for x in range(m)] for y in range(n)]
    for i in range(1,n):
        b[i][0] = "oben"
    for j in range(1,m):
        b[0][j] = "links"

    for i in range(1, n):
        for j in range(1, m):
            # to distinguish between a match/mismatch.
            match = False
            if l_v[i] == l_w[j]:
                s[i][j] = max(s[i - 1][j], s[i][j - 1], s[i - 1][j - 1] + 1)
                match = True
            else:
                s[i][j] = max(s[i - 1][j], s[i][j - 1])
            
            if s[i][j] == s[i - 1][j - 1] + 1 and match:
                b[i][j] = "oben-links"
            elif s[i][j] == s[i - 1][j]:
                b[i][j] = "oben"
            elif s[i][j] == s[i][j - 1]:
                b[i][j] = "links"

    return s[n-1][m-1], b


def print_matrix(matrix, title=None):
    if title:
        print(title)
    for line in matrix:
        print(line)
    print()


# Sequences
v = "ATGCG"
w = "ATCCGC"

# Get score s_nm and backtracking matrix B
score_nm, B = lgt(v, w)

# print score s[n][m]
print("Score:", score_nm)
print()

# print backtracking matrix
print_matrix(B, title='Backtracking b:')

Score: 4

Backtracking b:
['', 'links', 'links', 'links', 'links', 'links', 'links']
['oben', 'oben-links', 'links', 'links', 'links', 'links', 'links']
['oben', 'oben', 'oben-links', 'links', 'links', 'links', 'links']
['oben', 'oben', 'oben', 'oben', 'oben', 'oben-links', 'links']
['oben', 'oben', 'oben', 'oben-links', 'oben-links', 'oben', 'oben-links']
['oben', 'oben', 'oben', 'oben', 'oben', 'oben-links', 'oben']



# Aufgabe 1

In [34]:
def backtrack_v(B, v, i, j):
    if i == 0 and j == 0:
        return ""
    if i == 0:  
        return "-" + backtrack_v(B, v, i, j - 1)
    if j == 0:  
        return v[i - 1] + backtrack_v(B, v, i - 1, j)
    if B[i][j] == "oben-links":  
        return backtrack_v(B, v, i - 1, j - 1) + v[i - 1]
    elif B[i][j] == "oben":  
        return backtrack_v(B, v, i - 1, j) + v[i - 1]
    else:  
        return backtrack_v(B, v, i, j - 1) + "-"

In [50]:
assert backtrack_v(B, v, len(v), len(w)) == "AT--GCG"

In [36]:
def backtrack_w(B, w, i, j):
    if i == 0 or j == 0:
        return ""
    if B[i][j] == "oben-links":
        return backtrack_w(B, w, i - 1, j - 1) + w[j - 1]
    elif B[i][j] == "oben":
        return backtrack_w(B, w, i - 1, j) + "-"
    else:  # "links"
        return backtrack_w(B, w, i, j - 1) + w[j - 1]

In [48]:

assert backtrack_w(B, w, len(v), len(w)) == 'ATCCGC-'


# Aufgabe 2

In [51]:
def global_alignment(v, w, match_score=1, mismatch_score=0, gap_penalty=-1):
    
    # Sequence 1
    l_v = [""] + list(v)

    # Sequence 2
    l_w = [""] + list(w)

    n = len(l_v)
    m = len(l_w)

    # Initialize score matrix with gap penalties
    s = [[0 for x in range(m)] for y in range(n)]
    B = [["" for x in range(m)] for y in range(n)]

    for i in range(1, n):
        s[i][0] = gap_penalty * i
        B[i][0] = "oben" 
    for j in range(1, m):
        s[0][j] = gap_penalty * j
        B[0][j] = "links" 

    # Fill the DP matrix based on match, mismatch, and gap penalties
    for i in range(1, n):
        for j in range(1, m):
            if l_v[i] == l_w[j]:  # Match
                score_diag = s[i - 1][j - 1] + match_score
            else:  # Mismatch
                score_diag = s[i - 1][j - 1] + mismatch_score

            score_up = s[i - 1][j] + gap_penalty  
            score_left = s[i][j - 1] + gap_penalty

            # Take the maximum of the three possible movements
            s[i][j] = max(score_diag, score_up, score_left)

            if s[i][j] == score_diag:
                B[i][j] = "oben-links"  
            elif s[i][j] == score_up:
                B[i][j] = "oben"  
            else:
                B[i][j] = "links"  

    return s[n-1][m-1], B

In [57]:
def backtrack_global_alignment(B, v, w, i, j):
    aligned_v = []
    aligned_w = []
    
    while i > 0 or j > 0:
        if i > 0 and j > 0 and B[i][j] == "oben-links":
            aligned_v.append(v[i - 1])
            aligned_w.append(w[j - 1])
            i -= 1
            j -= 1
        elif i > 0 and (j == 0 or B[i][j] == "oben"):
            aligned_v.append(v[i - 1])
            aligned_w.append("-")
            i -= 1
        else:
            aligned_v.append("-")
            aligned_w.append(w[j - 1])
            j -= 1

    # Reverse the alignments
    return "".join(reversed(aligned_v)), "".join(reversed(aligned_w))


In [53]:
# Get score and backtracking matrix B for the global_alignment impl.
score_global, B_global = global_alignment(v, w, match_score=1, mismatch_score=0, gap_penalty=-1)

# print score s[n][m]
print("Score:", score_global)
print()

# print backtracking matrix
print_matrix(B_global, title='Backtracking B:')

Score: 3

Backtracking B:
['', 'links', 'links', 'links', 'links', 'links', 'links']
['oben', 'oben-links', 'links', 'links', 'links', 'links', 'links']
['oben', 'oben', 'oben-links', 'links', 'links', 'links', 'links']
['oben', 'oben', 'oben', 'oben-links', 'oben-links', 'oben-links', 'links']
['oben', 'oben', 'oben', 'oben-links', 'oben-links', 'links', 'oben-links']
['oben', 'oben', 'oben', 'oben', 'oben-links', 'oben-links', 'links']



In [58]:
aligned_v, aligned_w = backtrack_global_alignment(B_global, v, w, len(v), len(w))
print(f"Score: {score_global}")
print("LGT-Alignment:")
print(aligned_v)
print(aligned_w)

# Save to file
with open("alignment.txt", "w") as file:
    file.write(f"Score: {score_global}\n")
    file.write("LGT-Alignment:\n")
    file.write(aligned_v + "\n")
    file.write(aligned_w + "\n")

Score: 3
LGT-Alignment:
ATGCG-
ATCCGC


# Aufgabe 3

In [92]:
def lese_scoring_matrix(filename):
    """Read the scoring matrix from the blosum62.txt file and store it in a dictionary."""
    delta = {}
    
    with open(filename, 'r') as f:
        lines = f.readlines()

    # The first line contains the amino acids (columns), so we extract them
    amino_acids = []
    for line in lines:
        if line.startswith("#"):  # Skip comments
            continue
        if not amino_acids:
            # This should be the first valid line that contains amino acids
            amino_acids = line.split()
            continue

        # Process the actual matrix rows
        values = line.split()
        row_amino_acid = values[0]  # The first entry in the row is the amino acid
        scores = values[1:]  # The rest are the scores

        # Check that the number of scores matches the number of amino acids
        if len(scores) != len(amino_acids):
            print(f"Warning: Mismatch in row length for {row_amino_acid}")
            continue

        # Populate the dictionary with scores for each amino acid pair
        for j, score in enumerate(scores):
            try:
                # Handle '*' by assigning a default score or ignoring
                if score == '*':
                    delta[(row_amino_acid, amino_acids[j])] = -1  # Assign a default score for '*'
                else:
                    delta[(row_amino_acid, amino_acids[j])] = int(score)
            except ValueError:
                print(f"Skipping invalid score: {score} for pair ({row_amino_acid}, {amino_acids[j]})")

    return delta


In [95]:
def read_sequence(filename):
    """Read the amino acid sequence from a file and strip unwanted characters."""
    with open(filename, 'r') as f:
        return f.read().strip().replace('\n', '').replace(' ', '')

In [89]:
def needleman_wunsch(v, w, delta, indel_score=-5):    
    n = len(v)
    m = len(w)

    # Initialize DP matrix and backtracking matrix
    s = [[0 for _ in range(m + 1)] for _ in range(n + 1)]
    b = [["" for _ in range(m + 1)] for _ in range(n + 1)]

    # Initialize the first row and column with indel penalties
    for i in range(1, n + 1):
        s[i][0] = indel_score * i
        b[i][0] = "oben"
    for j in range(1, m + 1):
        s[0][j] = indel_score * j
        b[0][j] = "links"

    # Fill the DP matrix using the scoring matrix
    for i in range(1, n + 1):
        for j in range(1, m + 1):
            match = s[i - 1][j - 1] + delta[(v[i - 1], w[j - 1])]
            delete = s[i - 1][j] + indel_score
            insert = s[i][j - 1] + indel_score
            s[i][j] = max(match, delete, insert)

            if s[i][j] == match:
                b[i][j] = "oben-links"
            elif s[i][j] == delete:
                b[i][j] = "oben"
            else:
                b[i][j] = "links"

    return s[n][m], b

In [104]:
delta = lese_scoring_matrix("blosum62.txt")

# Load both the sequences
v = read_sequence("Homo_sapiens_hba.txt")
w = read_sequence("Homo_sapiens_mb.txt")

# Perform global alignment with indel score of -5
score, B = needleman_wunsch(v, w, delta, indel_score=-5)
aligned_v, aligned_w = backtrack_global_alignment(B, v, w, len(v), len(w))

print("Score: ", score)
print("aligned_v: ", aligned_v)
print("aligned_w: ",aligned_w)

Score:  106
aligned_v:  MVLSPADKTNVKAAWGKVGAHAGEYGAEALERMFLSFP-TTKTY--FPHF---DLSHGSAQVKGHGKKVADALTNAVAHVDDMPNA-LSALSDLHAHKLRVDPVNF-KLLSHCLLVTL-AAHLPAEF---TP-AVHASLDKFLASV-STVLTSKYR-
aligned_w:  MGLSDGEWQLVLNVWGKVEADIPGHGQEVLIRLFKGHPETLEKFDKFKHLKSEDEMKASEDLKKHGATVLTAL-GGILKKKGHHEAEIKPLAQSHATKHKI-PVKYLEFISECIIQVLQSKH-PGDFGADAQGAMNKALELFRKDMASNYKELGFQG


In [105]:
def backtrack_global_alignment(b, v, w, i, j):
    """Recursively backtrack to find the aligned sequences."""
    aligned_v = []
    aligned_w = []

    while i > 0 or j > 0:
        if i > 0 and j > 0 and b[i][j] == "oben-links":
            aligned_v.append(v[i - 1])
            aligned_w.append(w[j - 1])
            i -= 1
            j -= 1
        elif i > 0 and (j == 0 or b[i][j] == "oben"):
            aligned_v.append(v[i - 1])
            aligned_w.append("-")
            i -= 1
        else:
            aligned_v.append("-")
            aligned_w.append(w[j - 1])
            j -= 1

    # Reverse the alignments because we built them backwards
    return "".join(reversed(aligned_v)), "".join(reversed(aligned_w))

In [106]:
def write_alignment(score, aligned_v, aligned_w, filename):
    """Write the alignment and score to a file."""
    with open(filename, 'w') as f:
        f.write(f"Score: {score}\n")
        f.write(aligned_v + "\n")
        f.write(aligned_w + "\n")

In [109]:
write_alignment(score, aligned_v, aligned_w, "global_aln.txt")
print(f"Score with InDel = -5: {score}")

Score with InDel = -5: 106


In [110]:
 # Now, perform global alignment with indel score of 0
score_zero, B_zero = needleman_wunsch(v, w, delta, indel_score=0)
aligned_v_zero, aligned_w_zero = backtrack_global_alignment(B_zero, v, w, len(v), len(w))

# Write the result to another file
write_alignment(score_zero, aligned_v_zero, aligned_w_zero, "global_aln_indel_0.txt")

# Print the score for InDel = 0
print(f"Score with InDel = 0: {score_zero}")

Score with InDel = 0: 367
